# WDSR (2D)
---


# **How to use this notebook?**

---

<font size = 4>Video describing how to use ZeroCostDL4Mic notebooks are available on youtube:
  - [**Video 1**](https://www.youtube.com/watch?v=GzD2gamVNHI&feature=youtu.be): Full run through of the workflow to obtain the notebooks and the provided test datasets as well as a common use of the notebook
  - [**Video 2**](https://www.youtube.com/watch?v=PUuQfP5SsqM&feature=youtu.be): Detailed description of the different sections of the notebook


---
###**Structure of a notebook**

<font size = 4>The notebook contains two types of cell:  

<font size = 4>**Text cells** provide information and can be modified by douple-clicking the cell. You are currently reading the text cell. You can create a new text by clicking `+ Text`.

<font size = 4>**Code cells** contain code and the code can be modfied by selecting the cell. To execute the cell, move your cursor on the `[ ]`-mark on the left side of the cell (play button appears). Click to execute the cell. After execution is done the animation of play button stops. You can create a new coding cell by clicking `+ Code`.

---
###**Table of contents, Code snippets** and **Files**

<font size = 4>On the top left side of the notebook you find three tabs which contain from top to bottom:

<font size = 4>*Table of contents* = contains structure of the notebook. Click the content to move quickly between sections.

<font size = 4>*Code snippets* = contain examples how to code certain tasks. You can ignore this when using this notebook.

<font size = 4>*Files* = contain all available files. After mounting your google drive (see section 1.) you will find your files and folders here. 

<font size = 4>**Remember that all uploaded files are purged after changing the runtime.** All files saved in Google Drive will remain. You do not need to use the Mount Drive-button; your Google Drive is connected in section 1.2.

<font size = 4>**Note:** The "sample data" in "Files" contains default files. Do not upload anything in here!

---
###**Making changes to the notebook**

<font size = 4>**You can make a copy** of the notebook and save it to your Google Drive. To do this click file -> save a copy in drive.

<font size = 4>To **edit a cell**, double click on the text. This will show you either the source code (in code cells) or the source text (in text cells).
You can use the `#`-mark in code cells to comment out parts of the code. This allows you to keep the original code piece in the cell as a comment.

#**0. Before getting started**
---

<font size = 4>**We strongly recommend that you generate extra paired images. These images can be used to assess the quality of your trained model (Quality control dataset)**. The quality control assessment can be done directly in this notebook.

<font size = 4> **Additionally, the corresponding input and output files need to have the same name**.

<font size = 4> Please note that you currently can **only use .tif files!**


<font size = 4>Here's a common data structure that can work:
*   Experiment A
    - **Training dataset**
      - Low resolution (LR) images (Training_source)
        - img_1.tif, img_2.tif, ...
      - High resolution (HR) images (Training_target)
        - img_1.tif, img_2.tif, ...
    - **Quality control dataset**
     - Low resolution (LR) images
        - img_1.tif, img_2.tif
      - High resolution (HR) images
        - img_1.tif, img_2.tif
    - **Data to be predicted**
    - **Results**

---
<font size = 4>**Important note**

<font size = 4>- If you wish to **Train a network from scratch** using your own dataset (and we encourage everyone to do that), you will need to run **sections 1 - 4**, then use **section 5** to assess the quality of your model and **section 6** to run predictions using the model that you trained.

<font size = 4>- If you wish to **Evaluate your model** using a model previously generated and saved on your Google Drive, you will only need to run **sections 1 and 2** to set up the notebook, then use **section 5** to assess the quality of your model.

<font size = 4>- If you only wish to **run predictions** using a model previously generated and saved on your Google Drive, you will only need to run **sections 1 and 2** to set up the notebook, then use **section 6** to run the predictions on the desired model.
---

# **1. Install WDSR and dependencies**
---

In [ ]:
#@markdown ##Install Network and dependencies
Notebook_version = '1.13'
Network = 'RCAN (2D)'

%pip install tensorflow-addons --upgrade

import sys
sys.setrecursionlimit(10000)

import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
import numpy as np
import shutil
import time
import csv
import os

from matplotlib.pyplot import imread
from ipywidgets import interact
from glob import glob

from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity
from skimage.util import img_as_uint, img_as_float32
from skimage.util import random_noise
from skimage import transform
from skimage import filters
from skimage import metrics
from skimage import io

from sklearn.model_selection import train_test_split

from tensorflow.keras.layers import Add, Conv2D, Input, Lambda
from tensorflow.keras.models import Model

from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, Callback, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.image import ssim_multiscale as mssim

import tensorflow_addons as tfa

from keras.models import load_model
import keras

import logging
logging.getLogger('tensorflow').setLevel(logging.ERROR)

###

class bcolors:
    '''Definition of a color for the displays.'''
    WARNING = '\033[31m'

###

def set_seed(seed=56):
    ''' Sets the seed for the random operation in all 
        the packagees that use it.
    Args:
        seed (int): seed that will be established.
    '''
    np.random.seed(seed=seed)
    tf.random.set_seed(seed)
    os.environ["PYTHONHASHSEED"]=str(seed)

set_seed()

###

def em_crappify(img, scale):
    ''' Degradates and downscales the given image, simuating
        the degradation that a electron microscopy would suffer.
        Source https://www.biorxiv.org/content/10.1101/740548v3.
    Args:
        img (array): image that whose low resolution counterpart will be calcualted.
        scale (int): scale factor for the image downscaling. Example: 2.
    Returns:
        Low resolution image obtained from degradatin the received high resolution image.
    '''
    img = filters.gaussian(img, sigma=3) + 1e-6
    return transform.resize(img, (img.shape[0]//scale, img.shape[1]//scale), order=1)

def fluo_crappify(img,scale):
    ''' Degradates and downscales the given image, simuating
        the degradation that a fluorescence microscopy would suffer.
        Source https://www.biorxiv.org/content/10.1101/740548v3.
    Args:
        img (array): image that whose low resolution counterpart will be calcualted.
        scale (int): scale factor for the image downscaling. Example: 2.
    Returns:
        Low resolution image obtained from degradatin the received high resolution image.
    '''
    img = random_noise(img, mode='salt', amount=0.005)
    img = random_noise(img, mode='pepper', amount=0.005)
    img = filters.gaussian(img, sigma=5) + 1e-10
    return transform.resize(img, (img.shape[0]//scale, img.shape[1]//scale), order=1) 

def hr_to_lr(hr_img, down_factor, type_of_data):

    if type_of_data == "Electron microscopy":
        lr_img = em_crappify(hr_img, down_factor)
    else:
        lr_img = fluo_crappify(hr_img, down_factor)
    return lr_img

###

def calculate_down_factor (lr_imgs_basedir, hr_imgs_basedir):

    _, hr_extension = os.path.splitext(os.listdir(hr_imgs_basedir)[0])
    hr_filenames = [hr_imgs_basedir + '/' + x for x in os.listdir(hr_imgs_basedir) if x.endswith(hr_extension)]

    _, lr_extension = os.path.splitext(os.listdir(lr_imgs_basedir)[0])
    lr_filenames = [lr_imgs_basedir + '/' + x for x in os.listdir(lr_imgs_basedir) if x.endswith(lr_extension)]

    hr_img = io.imread(hr_filenames[0])
    lr_img = io.imread(lr_filenames[0])

    down_scale_x = hr_img.shape[0] / lr_img.shape[0]
    down_scale_y = hr_img.shape[1] / lr_img.shape[1]

    return [down_scale_x, down_scale_y]

###

def transf(lr, hr, horizontal_flip=False, vertical_flip=False,
              rotation=False):
    rng = np.random.default_rng()
    if horizontal_flip: 
        if rng.random() < 0.5:
            lr = tf.image.flip_left_right(lr)
            hr = tf.image.flip_left_right(hr)

    if vertical_flip:
        if rng.random() < 0.5:
            lr = tf.image.flip_up_down(lr)
            hr = tf.image.flip_up_down(hr)
    
    if rotation:
        if rng.random() < 0.5:
            k = rng.integers(3) + 1

            lr = tf.image.rot90(lr, k=k)
            hr = tf.image.rot90(hr, k=k)

    return  lr, hr


def generate_dataset(patch_size, down_factor, batch_size,
                     validation=False, validation_split=None,
                     hr_imgs_basedir="", lr_imgs_basedir="",
                     only_high_resolution_data=False,
                     only_hr_imgs_basedir="",
                     type_of_data="Electron microscopy",
                     horizontal_flip=False, vertical_flip=False,
                     rotation=False):

    data_generator = MicroscopyDataset(patch_size, patch_size, down_factor,
                                validation=validation, 
                                validation_split=validation_split, 
                                hr_imgs_basedir=hr_imgs_basedir, 
                                lr_imgs_basedir=lr_imgs_basedir,
                                only_high_resolution_data=only_high_resolution_data, 
                                only_hr_imgs_basedir=only_hr_imgs_basedir,
                                type_of_data=type_of_data)

    dataset = tf.data.Dataset.from_generator(data_generator, 
                                        output_types=(tf.float64,tf.float64))

    dataset = dataset.shuffle(np.round(data_generator.__len__()*0.2))
    dataset = dataset.batch(batch_size)
    if not validation:
      # Valdiation data will not suffer any transformation
      dataset = dataset.map(lambda lr, hr:transf(lr, hr, 
                                          horizontal_flip=horizontal_flip, 
                                          vertical_flip=vertical_flip,
                                          rotation=rotation))
    dataset = dataset.prefetch(tf.data.AUTOTUNE)

    return dataset, data_generator.__len__()

######

def create_random_patches( lr_path, hr_path, only_hr, type_hr_data,
                           only_hr_path, down_factor, lr_patch_size_width, 
                           lr_patch_size_height):
    ''' Create a list of images patches out of a list of images
    Args:
        lr_path (string): low resolution (LR) image path (input images).
        hr_path (string): high resolution (HR) image path (ground truth images).
        only_hr (boolean): indicates if only HR images will be used.
        type_hr_data (string): in case only HR images would be used, what type of data they would be (Electron microscopy or Fluorescence).
        only_hr_path (string): in case only HR images would be used, HR image path (ground truth images).
        down_factor (int): scale factor between LR and HR images. Example: 2.
        lr_patch_size_width (int): width of the LR patches.
        lr_patch_size_height (int): height of the LR patches.
        
    Returns:
        list of image patches (LR) and patches of corresponding labels (HR)
    '''
    if only_hr:
        used_hr_path = only_hr_path
    else: 
        used_hr_path = hr_path

    _, extension = os.path.splitext(os.listdir(used_hr_path)[0])
    filenames = [x for x in os.listdir(used_hr_path) if x.endswith(extension)]
    filenames.sort()

    input_patches = []
    output_patches = []
    for n in range(0, len(filenames)):
        hr_img = img_as_float32(io.imread( used_hr_path + '/' + filenames[n]))
        if only_hr:
            # If no path to the LR images is given, they will be artificially generated
            lr_img = hr_to_lr(hr_img, down_factor, type_hr_data)
        else:
            lr_img = img_as_float32(io.imread( lr_path + '/' + filenames[n]))
        
        if lr_patch_size_width is None:
            lr_patch_size_width = lr_img.shape[0]
        if lr_patch_size_height is None:
            lr_patch_size_height = lr_img.shape[1]

        for _ in range((lr_img.shape[0] // lr_patch_size_width)**2):
            lr_idx_width = np.random.randint(0, max(1, lr_img.shape[0] - lr_patch_size_width))
            lr_idx_height = np.random.randint(0, max(1, lr_img.shape[1] - lr_patch_size_height))
            hr_idx_width = lr_idx_width * down_factor
            hr_idx_height = lr_idx_height * down_factor
            
            input_patches.append(lr_img[lr_idx_width : lr_idx_width + lr_patch_size_width,
                                        lr_idx_height : lr_idx_height + lr_patch_size_height])
            output_patches.append(hr_img[hr_idx_width : hr_idx_width + lr_patch_size_width * down_factor,
                                        hr_idx_height : hr_idx_height + lr_patch_size_height * down_factor])

    return input_patches, output_patches

def create_complete_images( lr_path, hr_path, only_hr, 
                           type_hr_data, only_hr_path, down_factor):
    ''' Create a list of images patches out of a list of images
    Args:
        lr_path (string): low resolution (LR) image path (input images).
        hr_path (string): high resolution (HR) image path (ground truth images).
        only_hr (boolean): indicates if only HR images will be used.
        type_hr_data (string): in case only HR images would be used, what type of data they would be (Electron microscopy or Fluorescence).
        only_hr_path (string): in case only HR images would be used, HR image path (ground truth images).
        down_factor (int): scale factor between LR and HR images. Example: 2.
        
    Returns:
        list of image patches (LR) and patches of corresponding labels (HR)
    '''

    return create_random_patches(lr_path, hr_path, only_hr, type_hr_data,
                                only_hr_path, down_factor, None, None)

###

# Random rotation of an image by a multiple of 90 degrees
def random_90rotation( img ):
    return transform.rotate(img, 90*np.random.randint(0, 5), preserve_range=True)

# Runtime data augmentation
def get_train_val_generators(X_data, Y_data, validation_split=0.25,
                             batch_size=32, seed=42, show_examples=False,
                             rotation=True, horizontal_flip=True, vertical_flip=True ):
    X_train, X_test, Y_train, Y_test = train_test_split(X_data,
                                                        Y_data,
                                                        train_size=1-validation_split,
                                                        test_size=validation_split,
                                                        random_state=seed, shuffle=False)
    
    random_rotation=random_90rotation
    if not rotation:
      random_rotation=None

    # Image data generator distortion options
    data_gen_args = dict( preprocessing_function=random_rotation,
                          horizontal_flip=horizontal_flip,
                          vertical_flip=vertical_flip,
                          fill_mode='reflect')

    # Train data, provide the same seed and keyword arguments to the fit and flow methods
    X_datagen = ImageDataGenerator(**data_gen_args)
    Y_datagen = ImageDataGenerator(**data_gen_args)
    X_datagen.fit(X_train, augment=True, seed=seed)
    Y_datagen.fit(Y_train, augment=True, seed=seed)
    X_train_augmented = X_datagen.flow(X_train, batch_size=batch_size, shuffle=True, seed=seed)
    Y_train_augmented = Y_datagen.flow(Y_train, batch_size=batch_size, shuffle=True, seed=seed)
      
    
    # Validation data, no data augmentation, but we create a generator anyway
    X_datagen_val = ImageDataGenerator()
    Y_datagen_val = ImageDataGenerator()
    X_datagen_val.fit(X_test, augment=True, seed=seed)
    Y_datagen_val.fit(Y_test, augment=True, seed=seed)
    X_test_augmented = X_datagen_val.flow(X_test, batch_size=batch_size, shuffle=False, seed=seed)
    Y_test_augmented = Y_datagen_val.flow(Y_test, batch_size=batch_size, shuffle=False, seed=seed)
    
    if show_examples:
        plt.figure(figsize=(10,10))
        # generate samples and plot
        for i in range(9):
            # define subplot
            plt.subplot(330 + 1 + i)
            # generate batch of images
            batch = X_train_augmented.next()
            # convert to unsigned integers for viewing
            image = batch[0]
            # plot raw pixel data
            plt.imshow(image[:,:,0], vmin=0, vmax=1, cmap='gray')
        # show the figure
        plt.show()
        X_train_augmented.reset()
    
    # combine generators into one which yields image and masks
    train_generator = zip(X_train_augmented, Y_train_augmented)
    test_generator = zip(X_test_augmented, Y_test_augmented)
    
    return train_generator, test_generator


def subpixel_conv2d(scale):
    return lambda x: tf.nn.depth_to_space(x, scale)



def wdsr_a(scale, num_filters=32, num_res_blocks=8, res_block_expansion=4, res_block_scaling=None, out_channels=1):
    return wdsr(scale, num_filters, num_res_blocks, res_block_expansion, res_block_scaling, res_block_a, out_channels)


def wdsr_b(scale, num_filters=32, num_res_blocks=8, res_block_expansion=6, res_block_scaling=None, out_channels=1):
    return wdsr(scale, num_filters, num_res_blocks, res_block_expansion, res_block_scaling, res_block_b, out_channels)


def wdsr(scale, num_filters, num_res_blocks, res_block_expansion, res_block_scaling, res_block, out_channels=1):
    x_in = Input(shape=(None, None, out_channels))
    x = x_in
    # main branch
    m = conv2d_weightnorm(num_filters, 3, padding='same')(x)
    for i in range(num_res_blocks):
        m = res_block(m, num_filters, res_block_expansion, kernel_size=3, scaling=res_block_scaling)
    m = conv2d_weightnorm(out_channels * scale ** 2, 3, padding='same', name=f'conv2d_main_scale_{scale}')(m)
    m = Lambda(subpixel_conv2d(scale))(m)

    # skip branch
    s = conv2d_weightnorm(out_channels * scale ** 2, 5, padding='same', name=f'conv2d_skip_scale_{scale}')(x)
    s = Lambda(subpixel_conv2d(scale))(s)

    x = Add()([m, s])
    # final convolution with sigmoid activation ?
    #x = Conv2D(out_channels, 3, padding='same', activation='sigmoid')(x)

    return Model(x_in, x, name="wdsr")


def res_block_a(x_in, num_filters, expansion, kernel_size, scaling):
    x = conv2d_weightnorm(num_filters * expansion, kernel_size, padding='same', activation='relu')(x_in)
    x = conv2d_weightnorm(num_filters, kernel_size, padding='same')(x)
    if scaling:
        x = Lambda(lambda t: t * scaling)(x)
    x = Add()([x_in, x])
    return x


def res_block_b(x_in, num_filters, expansion, kernel_size, scaling):
    linear = 0.8
    x = conv2d_weightnorm(num_filters * expansion, 1, padding='same', activation='relu')(x_in)
    x = conv2d_weightnorm(int(num_filters * linear), 1, padding='same')(x)
    x = conv2d_weightnorm(num_filters, kernel_size, padding='same')(x)
    if scaling:
        x = Lambda(lambda t: t * scaling)(x)
    x = Add()([x_in, x])
    return x

def conv2d_weightnorm(filters, kernel_size, padding='same', activation=None, **kwargs):
    return tfa.layers.WeightNormalization(tf.keras.layers.Conv2D(filters, kernel_size, padding=padding, activation=activation, **kwargs), data_init=False)
######

class LearningRateObserver(Callback):

  def __init__(self):
    super(LearningRateObserver, self).__init__()
    self.epoch_lrs = []

  def on_epoch_end(self, epoch, logs=None):
    optimizer = self.model.optimizer
    lr = tf.keras.backend.eval(optimizer.lr)
    self.epoch_lrs.append(lr)

  def obtain_lrs(self):
    return self.epoch_lrs

######

class CosineAnnealer:
    
  def __init__(self, start, end, steps):
    self.start = start
    self.end = end
    self.steps = steps
    self.n = 0
      
  def step(self):
    self.n += 1
    cos = np.cos(np.pi * (self.n / self.steps)) + 1
    return self.end + (self.start - self.end) / 2. * cos


class OneCycleScheduler(Callback):
  """ `Callback` that schedules the learning rate on a 1cycle policy as per Leslie Smith's paper(https://arxiv.org/pdf/1803.09820.pdf).
  If the model supports a momentum parameter, it will also be adapted by the schedule.
  The implementation adopts additional improvements as per the fastai library: https://docs.fast.ai/callbacks.one_cycle.html, where
  only two phases are used and the adaptation is done using cosine annealing.
  In phase 1 the LR increases from `lr_max / div_factor` to `lr_max` and momentum decreases from `mom_max` to `mom_min`.
  In the second phase the LR decreases from `lr_max` to `lr_max / (div_factor * 1e4)` and momemtum from `mom_max` to `mom_min`.
  By default the phases are not of equal length, with the phase 1 percentage controlled by the parameter `phase_1_pct`.
  """

  def __init__(self, lr_max, steps, mom_min=0.85, mom_max=0.95, phase_1_pct=0.3, div_factor=25.):
    super(OneCycleScheduler, self).__init__()
    lr_min = lr_max / div_factor
    final_lr = lr_max / (div_factor * 1e4)
    phase_1_steps = steps * phase_1_pct
    phase_2_steps = steps - phase_1_steps
    
    self.phase_1_steps = phase_1_steps
    self.phase_2_steps = phase_2_steps
    self.phase = 0
    self.step = 0
    
    self.phases = [[CosineAnnealer(lr_min, lr_max, phase_1_steps), CosineAnnealer(mom_max, mom_min, phase_1_steps)], 
              [CosineAnnealer(lr_max, final_lr, phase_2_steps), CosineAnnealer(mom_min, mom_max, phase_2_steps)]]
    
    self.lrs = []
    self.moms = []

  def on_train_begin(self, logs=None):
    self.phase = 0
    self.step = 0

    self.set_lr(self.lr_schedule().start)
    self.set_momentum(self.mom_schedule().start)
      
  def on_train_batch_begin(self, batch, logs=None):
    self.lrs.append(self.get_lr())
    self.moms.append(self.get_momentum())

  def on_train_batch_end(self, batch, logs=None):
    self.step += 1
    if self.step >= self.phase_1_steps:
      self.phase = 1
        
    self.set_lr(self.lr_schedule().step())
    self.set_momentum(self.mom_schedule().step())
      
  def get_lr(self):
    try:
      return tf.keras.backend.get_value(self.model.optimizer.lr)
    except AttributeError:
      return None
      
  def get_momentum(self):
    try:
      return tf.keras.backend.get_value(self.model.optimizer.momentum)
    except AttributeError:
      return None
      
  def set_lr(self, lr):
    try:
      tf.keras.backend.set_value(self.model.optimizer.lr, lr)
    except AttributeError:
      pass # ignore
      
  def set_momentum(self, mom):
    try:
      tf.keras.backend.set_value(self.model.optimizer.momentum, mom)
    except AttributeError:
      pass # ignore

  def lr_schedule(self):
    return self.phases[self.phase][0]
  
  def mom_schedule(self):
    return self.phases[self.phase][1]
  
  def plot(self):
    ax = plt.subplot(1, 2, 1)
    ax.plot(self.lrs)
    ax.set_title('Learning Rate')
    ax = plt.subplot(1, 2, 2)
    ax.plot(self.moms)
    ax.set_title('Momentum')

#####


#--------------------- Display QC Maps --------------------------------

def visualise_image_comparison_QC(image, dimension, Source_folder, Prediction_folder, Ground_truth_folder, QC_folder, QC_scores):
  
  _, image_extension = os.path.splitext(image)

  if image_extension == '.tif':
    img_SSIM_GTvsSource = io.imread(os.path.join(QC_folder, 'SSIM_GTvsSource_'+image))
    img_SSIM_GTvsPrediction = io.imread(os.path.join(QC_folder, 'SSIM_GTvsPrediction_'+image))
    img_RSE_GTvsSource = io.imread(os.path.join(QC_folder, 'RSE_GTvsSource_'+image))
    img_RSE_GTvsPrediction = io.imread(os.path.join(QC_folder, 'RSE_GTvsPrediction_'+image))
  else:
    img_SSIM_GTvsSource = img_as_uint(io.imread(os.path.join(QC_folder, 'SSIM_GTvsSource_'+image)))/65536
    img_SSIM_GTvsPrediction = img_as_uint(io.imread(os.path.join(QC_folder, 'SSIM_GTvsPrediction_'+image)))/65536
    img_RSE_GTvsSource = img_as_uint(io.imread(os.path.join(QC_folder, 'RSE_GTvsSource_'+image)))/65536
    img_RSE_GTvsPrediction = img_as_uint(io.imread(os.path.join(QC_folder, 'RSE_GTvsPrediction_'+image)))/65536

  '''
  SSIM_GTvsP_forDisplay = QC_scores.loc[[image], 'Prediction v. GT mSSIM'].tolist()
  SSIM_GTvsS_forDisplay = QC_scores.loc[[image], 'Input v. GT mSSIM'].tolist()
  NRMSE_GTvsP_forDisplay = QC_scores.loc[[image], 'Prediction v. GT NRMSE'].tolist()
  NRMSE_GTvsS_forDisplay = QC_scores.loc[[image], 'Input v. GT NRMSE'].tolist()
  PSNR_GTvsP_forDisplay = QC_scores.loc[[image], 'Prediction v. GT PSNR'].tolist()
  PSNR_GTvsS_forDisplay = QC_scores.loc[[image], 'Input v. GT PSNR'].tolist()
  '''
 
  SSIM_GTvsP_forDisplay = QC_scores.loc[df['image #'] == image, 'Prediction v. GT mSSIM'].tolist()
  SSIM_GTvsS_forDisplay = QC_scores.loc[df['image #'] == image, 'Input v. GT mSSIM'].tolist()
  NRMSE_GTvsP_forDisplay = QC_scores.loc[df['image #'] == image, 'Prediction v. GT NRMSE'].tolist()
  NRMSE_GTvsS_forDisplay = QC_scores.loc[df['image #'] == image, 'Input v. GT NRMSE'].tolist()
  PSNR_GTvsP_forDisplay = QC_scores.loc[df['image #'] == image, 'Prediction v. GT PSNR'].tolist()
  PSNR_GTvsS_forDisplay = QC_scores.loc[df['image #'] == image, 'Input v. GT PSNR'].tolist()


  plt.figure(figsize=(15,15))

  # Source
  plt.subplot(3,3,1)
  plt.axis('off')
  img_Source = io.imread(os.path.join(Source_QC_folder, image))
  plt.imshow(img_Source,'gray')
  plt.title('Source',fontsize=15)

  # Target (Ground-truth)
  plt.subplot(3,3,2)
  plt.axis('off')
  img_GT = io.imread(os.path.join(Target_QC_folder,image))
  plt.imshow(img_GT ,'gray')
  plt.title('Target',fontsize=15)


  #Prediction
  plt.subplot(3,3,3)
  plt.axis('off')
  #img_Prediction = predictions
  img_Prediction = io.imread(os.path.join(prediction_QC_folder, image))
  plt.imshow(img_Prediction,'gray')
  plt.title('Prediction',fontsize=15)

  #Setting up colours
  cmap = plt.cm.CMRmap

  #SSIM between GT and Source
  plt.subplot(3,3,5)
  #plt.axis('off')
  plt.tick_params(
      axis='both',      # changes apply to the x-axis and y-axis
      which='both',      # both major and minor ticks are affected
      bottom=False,      # ticks along the bottom edge are off
      top=False,        # ticks along the top edge are off
      left=False,       # ticks along the left edge are off
      right=False,         # ticks along the right edge are off
      labelbottom=False,
      labelleft=False)   
  imSSIM_GTvsSource = plt.imshow(img_SSIM_GTvsSource, cmap = cmap, vmin=0, vmax=1)
  plt.colorbar(imSSIM_GTvsSource,fraction=0.046, pad=0.04)
  plt.title('Target vs. Source',fontsize=15)
  plt.xlabel('mSSIM: '+str(round(SSIM_GTvsS_forDisplay[0],3)),fontsize=14)
  plt.ylabel('SSIM maps',fontsize=20, rotation=0, labelpad=75)


  #SSIM between GT and Prediction
  plt.subplot(3,3,6)
  #plt.axis('off')
  plt.tick_params(
      axis='both',      # changes apply to the x-axis and y-axis
      which='both',      # both major and minor ticks are affected
      bottom=False,      # ticks along the bottom edge are off
      top=False,        # ticks along the top edge are off
      left=False,       # ticks along the left edge are off
      right=False,         # ticks along the right edge are off
      labelbottom=False,
      labelleft=False)  
  imSSIM_GTvsPrediction = plt.imshow(img_SSIM_GTvsPrediction, cmap = cmap, vmin=0,vmax=1)
  plt.colorbar(imSSIM_GTvsPrediction,fraction=0.046, pad=0.04)
  plt.title('Target vs. Prediction',fontsize=15)
  plt.xlabel('mSSIM: '+str(round(SSIM_GTvsP_forDisplay[0],3)),fontsize=14)


  #Root Squared Error between GT and Source
  plt.subplot(3,3,8)
  #plt.axis('off')
  plt.tick_params(
      axis='both',      # changes apply to the x-axis and y-axis
      which='both',      # both major and minor ticks are affected
      bottom=False,      # ticks along the bottom edge are off
      top=False,        # ticks along the top edge are off
      left=False,       # ticks along the left edge are off
      right=False,         # ticks along the right edge are off
      labelbottom=False,
      labelleft=False) 
  imRSE_GTvsSource = plt.imshow(img_RSE_GTvsSource, cmap = cmap, vmin=0, vmax = 1)
  plt.colorbar(imRSE_GTvsSource,fraction=0.046,pad=0.04)
  plt.title('Target vs. Source',fontsize=15)
  plt.xlabel('NRMSE: '+str(round(NRMSE_GTvsS_forDisplay[0],3))+', PSNR: '+str(round(PSNR_GTvsS_forDisplay[0],3)),fontsize=14)  
  plt.ylabel('RSE maps',fontsize=20, rotation=0, labelpad=75)

  #Root Squared Error between GT and Prediction
  plt.subplot(3,3,9)
  #plt.axis('off')
  plt.tick_params(
      axis='both',      # changes apply to the x-axis and y-axis
      which='both',      # both major and minor ticks are affected
      bottom=False,      # ticks along the bottom edge are off
      top=False,        # ticks along the top edge are off
      left=False,       # ticks along the left edge are off
      right=False,         # ticks along the right edge are off
      labelbottom=False,
      labelleft=False) 
  imRSE_GTvsPrediction = plt.imshow(img_RSE_GTvsPrediction, cmap = cmap, vmin=0, vmax=1)
  plt.colorbar(imRSE_GTvsPrediction,fraction=0.046,pad=0.04)
  plt.title('Target vs. Prediction',fontsize=15)
  plt.xlabel('NRMSE: '+str(round(NRMSE_GTvsP_forDisplay[0],3))+', PSNR: '+str(round(PSNR_GTvsP_forDisplay[0],3)),fontsize=14)
  #plt.savefig(QC_folder+"/QC_example_data.png",bbox_inches='tight',pad_inches=0)


# **2. Initialise the Colab session**
---







## **2.1. Check for GPU access**
---

By default, the session should be using Python 3 and GPU acceleration, but it is possible to ensure that these are set properly by doing the following:

<font size = 4>Go to **Runtime -> Change the Runtime type**

<font size = 4>**Runtime type: Python 3** *(Python 3 is programming language in which this program is written)*

<font size = 4>**Accelerator: GPU** *(Graphics processing unit)*


In [ ]:
#@markdown ##Run this cell to check if you have GPU access

if tf.test.gpu_device_name()=='':
  print('You do not have GPU access.') 
  print('Did you change your runtime ?') 
  print('If the runtime setting is correct then Google did not allocate a GPU for your session')
  print('Expect slow performance. To access GPU try reconnecting later')

else:
  print('You have GPU access')
  !nvidia-smi

## **2.2. Mount your Google Drive**
---
<font size = 4> To use this notebook on the data present in your Google Drive, you need to mount your Google Drive to this notebook.

<font size = 4> Play the cell below to mount your Google Drive and follow the link. In the new browser window, select your drive and select 'Allow', copy the code, paste into the cell and press enter. This will give Colab access to the data on the drive. 

<font size = 4> Once this is done, your data are available in the **Files** tab on the top left of notebook.

In [ ]:
#@markdown ##Run this cell to connect your Google Drive to Colab

#@markdown * Click on "Connect to Google Drive". 

#@markdown * Sign in your Google Account. 

#@markdown * Click on "Allow". 

#@markdown * Click on `Files` site on the right. Refresh the site. Your Google Drive folder should now be available here as `gdrive`. 

#mounts user's Google Drive to Google Colab.

from google.colab import drive
drive.mount('/content/gdrive')

# **3. Select your paths and parameters**

---


## **3.1. Setting the main training parameters**
---

<font size = 4>The code below allows the user to enter the paths to where the training data is and to define the training parameters. Note that the execution of the cell will take some time as the images from the folders will be read into memory.

<font size = 5> **Paths for training, predictions and results**

<font size = 4>**`Training_source`, `Training_target`:** These are the paths to your folders containing the Training_source and Training_target data respectively. To find the paths of the folders containing the respective datasets, go to your Files on the left of the notebook, navigate to the folder containing your files and copy the path by right-clicking on the folder, **Copy path** and pasting it into the right box below.
  
<font size = 4>**`Only_high_resolution_data`:** In case your dataset only contains high resolution data, he low resolution data will have to be generated artificially by a crappification function. Choose this if your dataset only contains high resolution data. **Default value: False**

<font size = 4>**`Type_of_data`:** In order to generate the low resolution data, it has to be known if the data is from *Electron microscopy* or *Fluorescence*. Based on [Deep Learning-Based Point-Scanning Super-Resolution Imaging](https://www.biorxiv.org/content/10.1101/740548v8) paper, this has to be known to be able of using the suitable crappification function.  **Default value: Electron microscopy**

<font size = 4>**`Down_factor`:** Scaling factor by which every dimension of the HR images is reduced. For example, if an HR image dimension is 256x256, and  its LR conterpart is 128x128, the Down_factor is 2. Typical values are 2, 4, etc. This is a **critical parameter** that depends on the acquisition of your LR and HR images. **Default value: 2**

<font size = 4>**`Training_high_resolution_folder`:** If *Only_high_resolution_data* has been selected, previous folder paths will not be operative and the path to the high resolution folder will have to be inserted here.

<font size = 4>**`Model_name`:** Use only my_model -style, not my-model (Use "_" not "-"). Do not use spaces in the name. Avoid using the name of an existing model (saved in the same folder) as it will be overwritten.

<font size = 4>**`Model_path`**: Enter the path where your model will be saved once trained (for instance your result folder).

<font size = 5>**Training parameters**

<font size = 4>**`Number_of_epochs`:** Input how many epochs (rounds) the network will be trained on. Since we use a fast-convergence algorithm, preliminary results can already be observed after 5-10 epochs, and full training could be achieved with as few as 15-20 epochs. Evaluate the  performance after training (see 5.). **Default value: 50**


<font size = 5>**Advanced parameters - experienced users only**

<font size =4>**`Batch_size:`** This parameter defines the number of patches seen in each training step. Reducing or increasing the **batch size** may slow or speed up your training, respectively, and can influence network performance. **Default value: 2**

<font size = 4>**`Percentage_validation`:**  Input the percentage of your training dataset you want to use to validate the network during training. **Default value: 10** 

<font size = 4>**`Initial_learning_rate`:**  Input the initial value to be used as learning rate. **Default value: 0.001**

<font size = 4>**`Source_patch_size`:** Size in both X and Y axis for the training patches that will be taken from each training image. A lower value would end up in a smaller patch and consequently in a faster training.  **Default value: 64**


In [ ]:
#@markdown ###Path to training images:

Training_source = "" #@param {type:"string"}
Training_target = "" #@param {type:"string"}

#@markdown #####In case you do not have paired LR and HR images, instead of filling the two parameters above completer the ones below: 

Only_high_resolution_data = False #@param {type:"boolean"}
Type_of_data = "Electron microscopy" #@param ["Electron microscopy", "Fluorescence"]
Training_high_resolution_folder = "" #@param {type:"string"}

Down_factor = 4 #@param {type:"number"}
if not Only_high_resolution_data:
    down_factor_x, down_factor_y = calculate_down_factor(Training_source, Training_target)
    if down_factor_x != down_factor_y:
        raise ValueError("Down factors on the width and height of the received images are not the same")
    else:
        Down_factor = down_factor_x
    if Down_factor % 1 !=0:
        raise ValueError("Down factor between hR and LR images is not a whole number.")
    else:
        Down_factor = int(Down_factor)

# Model name and path
#@markdown ###Names of both the model and path to the folder:
Model_name = "" #@param {type:"string"}
Model_path = "" #@param {type:"string"}

# Other parameters for training.
#@markdown ###Training Parameters

Number_of_epochs =  50#@param {type:"number"}

#@markdown ####Advanced Parameters

Use_Default_Advanced_Parameters = False #@param {type:"boolean"}
#@markdown ####If not, please input:

Batch_size =   1#@param {type:"number"}
Percentage_validation =  10 #@param {type:"number"}
Initial_learning_rate = 0.001 #@param {type:"number"}

Source_patch_size = 64 #@param {type:"number"}

if (Use_Default_Advanced_Parameters): 
  print("Default advanced parameters enabled")
  Batch_size = 2
  Percentage_validation = 10
  Initial_learning_rate = 0.001
  Source_patch_size = 64

# target_patch_size = source_patch_size * down_factor

#Here we define the percentage to use for validation
val_split = Percentage_validation/100


full_model_path = os.path.join(Model_path, Model_name)
if os.path.exists(full_model_path):
  print(bcolors.WARNING+'!! WARNING: Model folder already exists and will be overwritten if a model is trained !!')
  shutil.rmtree(full_model_path)
  
os.makedirs(full_model_path)
os.makedirs(os.path.join(full_model_path, 'Quality Control'))

## **3.2. Data augmentation**
---
<font size = 4>

<font size = 4>Data augmentation can improve training progress by amplifying differences in the dataset. This can be useful if the available dataset is small since, in this case, it is possible that a network could quickly learn every example in the dataset (overfitting), without augmentation. Augmentation is not necessary for training and if your training dataset is large you should disable it.

<font size = 4>Data augmentation is performed here by rotating the patches in XY-Plane and flip them along X-Axis and Y-Axis. This only works if the images are square in XY.


In [ ]:
#@markdown ###Data augmentation
Use_Data_augmentation = True #@param{type:"boolean"}

#@markdown Select this option if you want to use augmentation to increase the size of your dataset

#@markdown **Rotate each image randomly by 90 degrees.**
Rotation = True #@param{type:"boolean"}

#@markdown **Flip each image once around the x and y axis of the stack.**
Horizontal_flip = True #@param{type:"boolean"}
Vertical_flip = True #@param{type:"boolean"}


if Use_Data_augmentation:
  print("Data augmentation enabled")
else:
  print(bcolors.WARNING+"Data augmentation disabled")
  Rotation = False
  Horizontal_flip = False
  Vertical_flip = False

## **3.3. Using weights from a pre-trained model as initial weights**
---
<font size = 4>  Here, you can set the the path to a pre-trained model from which the weights can be extracted and used as a starting point for this training session. **This pre-trained model needs to be a WDSR model**. 

<font size = 4> This option allows you to perform training over multiple Colab runtimes or to do transfer learning using models trained outside of ZeroCostDL4Mic. **You do not need to run this section if you want to train a network from scratch**.

<font size = 4> In order to continue training from the point where the pre-trained model left off, it is adviseable to also **load the learning rate** that was used when the training ended. This is automatically saved for models trained with ZeroCostDL4Mic and will be loaded here. If no learning rate can be found in the model folder provided, the default learning rate will be used. 

In [ ]:
# @markdown ##Loading weights from a pre-trained network

Use_pretrained_model = False #@param {type:"boolean"}

Weights_choice = "last" #@param ["last", "best"]


#@markdown ###If you do, please provide the path to the model folder:
Pretrained_model_folder = "" #@param {type:"string"}

# --------------------- Check if we load a previously trained model ------------------------
if not Use_pretrained_model:
  print(bcolors.WARNING+'No pretrained network will be used.')
else:
  h5_file_path = os.path.join(Pretrained_model_folder, "weights_"+Weights_choice+".h5")


# --------------------- Check the model exist ------------------------
# If the model path chosen does not contain a pretrain model then use_pretrained_model is disabled, 
  if not os.path.exists(h5_file_path):
    print(bcolors.WARNING+'WARNING: weights_'+Weights_choice+'.h5 pretrained model does not exist')
    Use_pretrained_model = False
  else:
# If the model path contains a pretrain model, we load the training rate, 
    print('Weights found in:')
    print(h5_file_path)
    print('will be loaded prior to training.')
  
    if os.path.exists(os.path.join(Pretrained_model_folder, 'Quality Control', 'training_evaluation.csv')):

      with open(os.path.join(Pretrained_model_folder, 'Quality Control', 'training_evaluation.csv'),'r') as csvfile:
        csvRead = pd.read_csv(csvfile, sep=',')
        #print(csvRead)
    
        if "learning rate" in csvRead.columns: #Here we check that the learning rate column exist (compatibility with model trained un ZeroCostDL4Mic bellow 1.4)
          print("\nPretrained network learning rate found.")
          #find the last learning rate
          lastLearningRate = csvRead["learning rate"].iloc[-1]
          #Find the learning rate corresponding to the lowest validation loss
          min_val_loss = csvRead[csvRead['val_loss'] == min(csvRead['val_loss'])]
          #print(min_val_loss)
          bestLearningRate = min_val_loss['learning rate'].iloc[-1]

          if Weights_choice == "last":
            print('Last learning rate: '+str(lastLearningRate))

          if Weights_choice == "best":
            print('Learning rate of best validation loss: '+str(bestLearningRate))

        if not "learning rate" in csvRead.columns: #if the column does not exist, then initial learning rate is used instead
          bestLearningRate = Initial_learning_rate
          lastLearningRate = Initial_learning_rate
          print(bcolors.WARNING+'WARNING: The learning rate cannot be identified from the pretrained network. Default learning rate of '+str(bestLearningRate)+' will be used instead' + bcolors.W)

#Compatibility with models trained outside ZeroCostDL4Mic but default learning rate will be used
    if not os.path.exists(os.path.join(Pretrained_model_folder, 'Quality Control', 'training_evaluation.csv')):
      print(bcolors.WARNING+'WARNING: The learning rate cannot be identified from the pretrained network. Default learning rate of '+str(initial_learning_rate)+' will be used instead'+ bcolors.W)
      bestLearningRate = Initial_learning_rate
      lastLearningRate = Initial_learning_rate


# **4. Train the network**
---

## **4.1. Prepare the data and model for training**
---
<font size = 4>Here, we use the information from section 3 to build the model and convert the training data into a suitable format for training. A pair or LR-HR training images will be displayed at the end of the process.

In [ ]:
#@markdown ##Play this cell to prepare the model for training

# Read the list of file names, same for source and target

train_patches_wf, train_patches_gt = create_random_patches(Training_source,
                                                           Training_target,
                                                           Only_high_resolution_data,
                                                           Type_of_data,
                                                           Training_high_resolution_folder,
                                                           Down_factor, 
                                                           Source_patch_size,
                                                           Source_patch_size) 

# display patch

plt.figure(figsize=(10,5))
plt.subplot(1, 2, 1)
plt.imshow( train_patches_wf[0], 'gray' )
plt.title( 'Training patch at low resolution' )
# Side by side with its "ground truth"
plt.subplot(1, 2, 2)
plt.imshow( train_patches_gt[0], 'gray' )
plt.title( 'Training patch at full resolution' )
plt.savefig('/content/ExampleData.png', bbox_inches='tight', pad_inches=0)

# Prepare the training data and create data generators
# training input
X_train = train_patches_wf.copy()
X_train = np.expand_dims(X_train, axis=-1)
# training ground truth
Y_train = train_patches_gt.copy()
Y_train = np.expand_dims(Y_train, axis=-1)

# Calculate the number of steps to use for validation
validation_steps = np.ceil(len(X_train)*val_split/Batch_size)

# define data generators
train_generator, val_generator = get_train_val_generators(X_data=X_train,
                                                          Y_data=Y_train,
                                                          validation_split=val_split,
                                                          batch_size=Batch_size,
                                                          show_examples=False,
                                                          rotation=Rotation,
                                                          horizontal_flip=Horizontal_flip,
                                                          vertical_flip=Vertical_flip)

# This modelcheckpoint will only save the best model from the validation loss point of view
model_checkpoint = ModelCheckpoint(os.path.join(full_model_path, 'weights_best.h5'), 
                                   monitor='val_loss',verbose=1, 
                                   save_best_only=True, save_weights_only=True)

lr_observer = LearningRateObserver()

# --------------------- Using pretrained model ------------------------
#Here we ensure that the learning rate set correctly when using pre-trained models
if 'Use_pretrained_model' in locals():
  if Use_pretrained_model:
    if Weights_choice == "last":
      Initial_learning_rate = lastLearningRate

    if Weights_choice == "best":            
      Initial_learning_rate = bestLearningRate
  else:
    h5_file_path = None
else:
  Use_pretrained_model = False

input_shape = X_train.shape[1:]
output_shape = Y_train.shape[1:]
print("Input patch shape: " + str(input_shape))
print("Output patch shape: " + str(output_shape))

# ----------------------DEFINE THE MODEL----------------------------

# Custom WDSR B model (0.62M parameters)
model = wdsr_b(scale=Down_factor, num_res_blocks=32)

optim = tf.keras.optimizers.Adam( learning_rate=Initial_learning_rate )

loss_funct = 'mean_absolute_error' # [ms_ssim_loss(max_val=1)]
eval_metric = 'mean_squared_error'

# compile the model with RMSprop as optimizer, MSE as loss function and MAE as metric
model.compile(optimizer=optim, loss=loss_funct, metrics=[eval_metric])

if 'Use_pretrained_model' in locals():
  if Use_pretrained_model:
    	model.load_weights(h5_file_path)

# callback for early stop
earlystopper = EarlyStopping(patience=10, verbose=1, restore_best_weights=True)

# --------------------- ---------------------- ------------------------

steps = np.ceil(len(X_train) / Batch_size) * Number_of_epochs
lr_schedule = OneCycleScheduler(Initial_learning_rate, steps)

# ------------------ Failsafes ------------------

full_model_path = os.path.join(Model_path, Model_name)
if os.path.exists(full_model_path):
  print(bcolors.WARNING+'!! WARNING: Model folder already existed and has been removed !!'+ bcolors.WARNING)
  shutil.rmtree(full_model_path)
os.makedirs(full_model_path)
os.makedirs(os.path.join(full_model_path,'Quality Control'))

# ------------------ Display ------------------
print('---------------------------- Main training parameters ----------------------------')
print('Number of epochs: '+str(Number_of_epochs))
print('Batch size: '+str(Batch_size))
print('Scaling factor: '+str(Down_factor))
#print('Number of training images: '+str(number_of_patches))
#print('Number of training steps: '+str(steps))
#print('Number of validation steps: '+str(validation_steps))
print('---------------------------- ------------------------ ----------------------------')


## **4.2. Start Training**
---
<font size = 4>When playing the cell below you should see updates after each epoch (round). Network training can take some time.

<font size = 4>* **CRITICAL NOTE:** Google Colab has a time limit for processing (to prevent using GPU power for datamining). Training time must be less than 12 hours! If training takes longer than 12 hours, please decrease the number of epochs or number of patches.

<font size = 4>Once training is complete, the trained model is automatically saved on your Google Drive, in the **Model_path** folder that was selected in Section 3. It is however wise to download the folder as all data can be erased at the next training if using the same folder.

In [ ]:
#@markdown ##Start training

# Export the training parameters as pdf (before training, in case training fails) 
#pdf_export(augmentation = Use_Data_augmentation, pretrained_model = Use_pretrained_model)

start = time.time()

# Start Training
history = model.fit(train_generator, validation_data=val_generator,
                      validation_steps=validation_steps,
                      steps_per_epoch=np.ceil(len(X_train)/Batch_size),
                      epochs=Number_of_epochs, 
                      callbacks=[model_checkpoint, lr_schedule, lr_observer])

#model.save(os.path.join(full_model_path, 'last_model.h5'))
model.save_weights(os.path.join(full_model_path, 'weights_last.h5'))

#Note that the notebook should load weights either from the model that is 
#trained from scratch or if the pretrained weights are used (3.3.)

# convert the history.history dict to a pandas DataFrame:     
lossData = pd.DataFrame(history.history) 

lr_each_epoch = lr_observer.obtain_lrs()

# The training evaluation.csv is saved (overwrites the Files if needed). 
lossDataCSVpath = os.path.join(full_model_path,'Quality Control/training_evaluation.csv')
with open(lossDataCSVpath, 'w') as f:
  writer = csv.writer(f)
  writer.writerow(['loss','mean_squared_error','val_loss', 'val_mean_squared_error', 'learning rate'])
  for i in range(len(history.history['loss'])):
    writer.writerow([history.history['loss'][i], history.history['mean_squared_error'][i] , history.history['val_loss'][i], history.history['val_mean_squared_error'][i], lr_each_epoch[i]])
    
# Displaying the time elapsed for training
dt = time.time() - start
mins, sec = divmod(dt, 60) 
hour, mins = divmod(mins, 60) 
print("Time elapsed:",hour, "hour(s)",mins,"min(s)",round(sec),"sec(s)")


# **5. Evaluate your model**
---

<font size = 4>This section allows the user to perform important quality checks on the validity and generalisability of the trained model. 

<font size = 4>**We highly recommend to perform quality control on all newly trained models.**



In [ ]:
# model name and path
#@markdown ###Do you want to assess the model you just trained ?
Use_the_current_trained_model = True #@param {type:"boolean"}

#@markdown ###If not, please provide the path to model folder and the scale between the low and high resolution images:

QC_model_folder = "" #@param {type:"string"}
QC_down_factor =  2 #@param {type:"number"}

#Here we define the loaded model name and path
QC_model_name = os.path.basename(QC_model_folder)
QC_model_path = os.path.dirname(QC_model_folder)

if (Use_the_current_trained_model): 
  QC_model_name = Model_name
  QC_model_path = Model_path
else:
  Down_factor = QC_down_factor
  Initial_learning_rate = 0.001

full_QC_model_path = QC_model_path+'/'+QC_model_name

#print(full_QC_model_path)

if os.path.exists(full_QC_model_path):
  print("The "+QC_model_name+" network will be evaluated")
else:
  print(bcolors.WARNING+'!! WARNING: The chosen model does not exist !!'+bcolors.WARNING)
  print('Please make sure you provide a valid model path and model name before proceeding further.')


## **5.1. Inspection of the loss function**
---

<font size = 4>First, it is good practice to evaluate the training progress by comparing the training loss with the validation loss. The latter is a metric which shows how well the network performs on a subset of unseen data which is set aside from the training dataset. For more information on this, see for example [this review](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6381354/) by Nichols *et al.*

<font size = 4>**Training loss** describes an error value after each epoch for the difference between the model's prediction and its ground-truth target.

<font size = 4>**Validation loss** describes the same error value between the model's prediction on a validation image and compared to it's target.

<font size = 4>During training both values should decrease before reaching a minimal value which does not decrease further even after more training. Comparing the development of the validation loss with the training loss can give insights into the model's performance.

<font size = 4>Decreasing **Training loss** and **Validation loss** indicates that training is still necessary and increasing the `Number_of_epochs` is recommended. Note that the curves can look flat towards the right side, just because of the y-axis scaling. The network has reached convergence once the curves flatten out. After this point no further training is required. If the **Validation loss** suddenly increases again an the **Training loss** simultaneously goes towards zero, it means that the network is overfitting to the training data. In other words the network is remembering the exact patterns from the training data and no longer generalizes well to unseen data. In this case the training dataset has to be increased.

In [ ]:
#@markdown ##Play the cell to show a plot of training errors vs. epoch number

lossDataFromCSV = []
vallossDataFromCSV = []

mseDataFromCSV = []
valmseDataFromCSV = []

csv_path = QC_model_path+'/'+QC_model_name+'/Quality Control/training_evaluation.csv'

if os.path.exists(csv_path):
  with open(QC_model_path+'/'+QC_model_name+'/Quality Control/training_evaluation.csv','r') as csvfile:
    csvRead = csv.reader(csvfile, delimiter=',')
    next(csvRead)
    for row in csvRead:
        lossDataFromCSV.append(float(row[0]))
        vallossDataFromCSV.append(float(row[2]))
        mseDataFromCSV.append(float(row[1]))
        valmseDataFromCSV.append(float(row[3]))

  epochNumber = range(len(lossDataFromCSV))
  plt.figure(figsize=(15,10))

  plt.subplot(2,1,1)
  plt.plot(epochNumber,lossDataFromCSV, label='Training loss')
  plt.plot(epochNumber,vallossDataFromCSV, label='Validation loss')
  plt.title('Training loss and validation loss vs. epoch number')
  plt.ylabel('Loss')
  plt.xlabel('Epoch number')
  plt.legend()

  plt.subplot(2,1,2)
  plt.plot(epochNumber,mseDataFromCSV, label='Training MSE')
  plt.plot(epochNumber,valmseDataFromCSV, label='Validation MSE')
  plt.title('Training MSE and validation MSE vs. epoch number')
  plt.ylabel('Loss')
  plt.xlabel('Epoch number')
  plt.legend()
  plt.savefig(QC_model_path+'/'+QC_model_name+'/Quality Control/lossCurvePlots.png')
  plt.show()
else:
  print(bcolors.WARNING+'The path does not contain a csv file containing the loss and validation evolution of the model'+bcolors.WARNING)

## **5.2. Image predictions**
---


In [ ]:
#@markdown ##Choose the folders that contain your Quality Control dataset

Source_QC_folder = "" #@param{type:"string"}
Target_QC_folder = "" #@param{type:"string"}

#@markdown #####In case you do not have paired LR and HR images, instead of filling the two parameters above completer the ones below: 

Only_high_resolution_data = False #@param {type:"boolean"}
Type_of_data = "Electron microscopy" #@param ["Electron microscopy", "Fluorescence"]
QC_high_resolution_folder = "" #@param {type:"string"}

# Insert code to perform predictions on all datasets in the Source_QC folder

X_test, Y_test = create_complete_images( Source_QC_folder, Target_QC_folder, 
                                        Only_high_resolution_data, 
                                        Type_of_data, 
                                        QC_high_resolution_folder, 
                                        Down_factor)

if Only_high_resolution_data:
  Source_QC_folder = "/content/LR_images"
  Target_QC_folder = QC_high_resolution_folder

_, test_extension = os.path.splitext(os.listdir(Target_QC_folder)[0])

test_filenames = [x for x in os.listdir( Target_QC_folder ) if x.endswith(test_extension)]
test_filenames.sort()

print( 'Available images : ' + str( len(test_filenames)) )

if Only_high_resolution_data:
  if os.path.exists(Source_QC_folder):
    shutil.rmtree(Source_QC_folder)
  os.makedirs(Source_QC_folder)
  for i, image in enumerate(np.expand_dims(X_test, axis=-1)):
    tf.keras.preprocessing.image.save_img(Source_QC_folder+'/'+test_filenames[i], image, data_format=None, file_format=None)

X_test = np.expand_dims(X_test, axis=-1)
Y_test = np.expand_dims(Y_test, axis=-1)


# Custom WDSR B model (0.62M parameters)
model = wdsr_b(scale=Down_factor, num_res_blocks=32)

optim = tf.keras.optimizers.Adam( learning_rate=Initial_learning_rate )

loss_funct = 'mean_absolute_error' # [ms_ssim_loss(max_val=1)]
eval_metric = 'mean_squared_error'

# compile the model with RMSprop as optimizer, MSE as loss function and MAE as metric
model.compile(optimizer=optim, loss=loss_funct, metrics=[eval_metric])

# Load weights
if (Use_the_current_trained_model): 
  model.load_weights( os.path.join(full_model_path, 'weights_best.h5') )
else:
  model.load_weights( os.path.join(full_QC_model_path, 'weights_best.h5') )


# Evaluate the model on the test data using evaluate
print( 'Evaluating...' )
results = model.evaluate(X_test, Y_test , batch_size=2)
print('The results are:')
print('  Loss: '+ str(results[0]))
print('  MSE: '+ str(results[1]))

print( 'Predicting...' )
predictions = model.predict(X_test, batch_size=8)

#Save the predictions
prediction_QC_folder = os.path.join(full_QC_model_path, 'Quality Control', 'Prediction')
if os.path.exists(prediction_QC_folder):
  shutil.rmtree(prediction_QC_folder)
os.makedirs(prediction_QC_folder)


for i, image in enumerate(predictions):
  tf.keras.preprocessing.image.save_img(prediction_QC_folder+'/'+test_filenames[i], image, data_format=None, file_format=None)


# ------------- For display ------------
print('------------------------------------------------------------------------------------------------------------------------------------------------------------------------------')
print("Choose the image file (Once selected, the new set of images will take a few seconds to appear):")
print(" ")
@interact
def show_prediction_results(file = sorted(os.listdir(prediction_QC_folder))):
  imageLR = io.imread(os.path.join(Source_QC_folder, file))
  imageHR = io.imread(os.path.join(Target_QC_folder, file))
  imageP = io.imread(os.path.join(prediction_QC_folder, file))

  plt.figure(figsize=(25,25))
  plt.subplot(3, 3, 1)
  plt.imshow( imageLR, 'gray' )
  plt.title( 'Low resolution' )
  # Side by side with its "ground truth"
  plt.subplot(3, 3, 2)
  plt.imshow( imageHR, 'gray' )
  plt.title( 'High resolution' )
  # ant its prediction
  plt.subplot(3, 3, 3)
  plt.imshow( imageP, 'gray' )
  plt.title( 'Prediction' ) 

print('------------------------------------------------------------------------------------------------------------------------------------------------------------------------------')


## **5.3. Error mapping and quality metrics estimation**
---


<font size = 4>This section will calculate the SSIM, PNSR and MSSIM metrics between the predicted and target images to evaluate the quality of the results.

<font size = 4>**1. The SSIM (structural similarity) map** 

<font size = 4>The SSIM metric is used to evaluate whether two images contain the same structures. It is a normalized metric and an SSIM of 1 indicates a perfect similarity between two images. Therefore for SSIM, the closer to 1, the better.
<font size=4>**mSSIM** is the SSIM value calculated across the entire window of both images.

<font size = 4>**2. PSNR (Peak signal-to-noise ratio)** is a metric that gives the difference between the ground truth and prediction (or source input) in decibels, using the peak pixel values of the prediction and the MSE between the images. The higher the score the better the agreement.





In [ ]:
#@markdown ##Run to calculate the PSNR, SSIM and MSSIM metrics

#Now, we calculate the image metrics

_MSSSIM_WEIGHTS = (0.0448, 0.2856, 0.3001, 0.2363, 0.1333)

psnr_array = []
ssim_array = []
mssim_array = []

for i in range(0, len(predictions)):
  pred = np.clip( predictions[i][:,:,0], a_min=0, a_max=1 )
  psnr_array.append(metrics.peak_signal_noise_ratio(pred, Y_test[i][:,:,0]))
  ssim_array.append(metrics.structural_similarity(pred, Y_test[i][:,:,0]))

  pred = np.clip( predictions[i][:,:,:], a_min=0, a_max=1 )
  mssim_array.append(mssim(pred, Y_test[i][:,:,:].astype('float32'), max_val=1, power_factors=_MSSSIM_WEIGHTS, filter_size=11,
    filter_sigma=1.5, k1=0.01, k2=0.03 ))
  
psnr_mean = np.mean(psnr_array)
ssim_mean = np.mean(ssim_array)
mssim_mean = np.mean(mssim_array)

print("PSNR: ", psnr_mean)
print("SSIM: ", ssim_mean)
print("MSSIM:", mssim_mean)


<font size = 4>The metrics shown in the previous cell are an average of all the test dataset. In the following cell the metrics refer only to the displayed image. You can see all the metrics for each image in a csv file saved in the Quality Control folder.

In [ ]:
#@markdown ##SSIM and RSE map
#@markdown Now we will show the SSIM and RSE maps between the original images (upsampled using simple interpolation) and the target images, together with the maps between the predicted and target images.

def ssim(img1, img2):
  return structural_similarity(img1,img2,data_range=1.,full=True, gaussian_weights=True, use_sample_covariance=False, sigma=1.5)


def normalize(x, pmin=3, pmax=99.8, axis=None, clip=False, eps=1e-20, dtype=np.float32):
    """This function is adapted from Martin Weigert"""
    """Percentile-based image normalization."""

    mi = np.percentile(x,pmin,axis=axis,keepdims=True)
    ma = np.percentile(x,pmax,axis=axis,keepdims=True)
    return normalize_mi_ma(x, mi, ma, clip=clip, eps=eps, dtype=dtype)


def normalize_mi_ma(x, mi, ma, clip=False, eps=1e-20, dtype=np.float32):#dtype=np.float32
    """This function is adapted from Martin Weigert"""
    if dtype is not None:
        x   = x.astype(dtype,copy=False)
        mi  = dtype(mi) if np.isscalar(mi) else mi.astype(dtype,copy=False)
        ma  = dtype(ma) if np.isscalar(ma) else ma.astype(dtype,copy=False)
        eps = dtype(eps)

    try:
        import numexpr
        x = numexpr.evaluate("(x - mi) / ( ma - mi + eps )")
    except ImportError:
        x =(x - mi) / ( ma - mi + eps )

    if clip:
        x = np.clip(x,0,1)

    return x

def norm_minmse(gt, x, normalize_gt=True):
    """This function is adapted from Martin Weigert"""

    """
    normalizes and affinely scales an image pair such that the MSE is minimized  
     
    Parameters
    ----------
    gt: ndarray
        the ground truth image      
    x: ndarray
        the image that will be affinely scaled 
    normalize_gt: bool
        set to True of gt image should be normalized (default)
    Returns
    -------
    gt_scaled, x_scaled 
    """
    if normalize_gt:
        gt = normalize(gt, 0.1, 99.9, clip=False).astype(np.float32, copy = False)
    x = x.astype(np.float32, copy=False) - np.mean(x)
    #x = x - np.mean(x)
    gt = gt.astype(np.float32, copy=False) - np.mean(gt)
    #gt = gt - np.mean(gt)
    x_scaled = transform.resize( x, (gt.shape[0], gt.shape[1]), order=1 )
    scale = np.cov(x_scaled.flatten(), gt.flatten())[0, 1] / np.var(x_scaled.flatten())
    return gt, scale * x_scaled
    #return gt, x_scaled


# Open and create the csv file that will contain all the QC metrics
with open(QC_model_path+"/"+QC_model_name+"/Quality Control/QC_metrics_"+QC_model_name+".csv", "w", newline='') as file:
    writer = csv.writer(file)

    # Write the header in the csv file
    writer.writerow(["image #","Prediction v. GT mSSIM","Input v. GT mSSIM", "Prediction v. GT NRMSE", "Input v. GT NRMSE", "Prediction v. GT PSNR", "Input v. GT PSNR"])  

    # Let's loop through the provided dataset in the QC folders
    print('Computing maps...')

    _, image_extension = os.path.splitext(os.listdir(Target_QC_folder)[0])

    for i in os.listdir(Target_QC_folder):
      if not os.path.isdir(os.path.join(Source_QC_folder,i)) and i.endswith(image_extension):

      # -------------------------------- Target test data (Ground truth) --------------------------------
        test_GT = io.imread(os.path.join(Target_QC_folder, i))
        #test_GT = test_patches_gt

      # -------------------------------- Source test data --------------------------------
        test_source = io.imread(os.path.join(Source_QC_folder,i))
        #test_source = test_patches_wf

      # Normalize the images wrt each other by minimizing the MSE between GT and Source image
        test_GT_norm,test_source_norm = norm_minmse(test_GT, test_source, normalize_gt=True)

      # -------------------------------- Prediction --------------------------------
        test_prediction = io.imread(os.path.join(prediction_QC_folder,i))

      # Normalize the images wrt each other by minimizing the MSE between GT and prediction
        test_GT_norm,test_prediction_norm = norm_minmse(test_GT, test_prediction, normalize_gt=True)        

      # -------------------------------- Calculate the metric maps and save them --------------------------------

      # Calculate the SSIM maps
        index_SSIM_GTvsPrediction, img_SSIM_GTvsPrediction = ssim(test_GT_norm, test_prediction_norm)
        index_SSIM_GTvsSource, img_SSIM_GTvsSource = ssim(test_GT_norm, test_source_norm)

      #Save ssim_maps
        img_SSIM_GTvsPrediction_32bit = np.float32(img_SSIM_GTvsPrediction)
        img_SSIM_GTvsSource_32bit = np.float32(img_SSIM_GTvsSource)
        if image_extension == '.tif':
          io.imsave(QC_model_path+'/'+QC_model_name+'/Quality Control/SSIM_GTvsPrediction_'+i, img_SSIM_GTvsPrediction_32bit)
          io.imsave(QC_model_path+'/'+QC_model_name+'/Quality Control/SSIM_GTvsSource_'+i, img_SSIM_GTvsSource_32bit)
        else:
          io.imsave(QC_model_path+'/'+QC_model_name+'/Quality Control/SSIM_GTvsPrediction_'+i, img_as_uint(img_SSIM_GTvsPrediction_32bit))
          io.imsave(QC_model_path+'/'+QC_model_name+'/Quality Control/SSIM_GTvsSource_'+i, img_as_uint(img_SSIM_GTvsSource_32bit))
            
      # Calculate the Root Squared Error (RSE) maps
        img_RSE_GTvsPrediction = np.sqrt(np.square(test_GT_norm - test_prediction_norm))
        img_RSE_GTvsSource = np.sqrt(np.square(test_GT_norm - test_source_norm))

      # Save SE maps
        img_RSE_GTvsPrediction_32bit = np.float32(img_RSE_GTvsPrediction)
        img_RSE_GTvsSource_32bit = np.float32(img_RSE_GTvsSource)
        if image_extension == '.tif':
          io.imsave(QC_model_path+'/'+QC_model_name+'/Quality Control/RSE_GTvsPrediction_'+i, np.clip(img_RSE_GTvsPrediction_32bit, -1, 1))
          io.imsave(QC_model_path+'/'+QC_model_name+'/Quality Control/RSE_GTvsSource_'+i, np.clip(img_RSE_GTvsSource_32bit, -1, 1))
        else:
          io.imsave(QC_model_path+'/'+QC_model_name+'/Quality Control/RSE_GTvsPrediction_'+i, img_as_uint(np.clip(img_RSE_GTvsPrediction_32bit, -1, 1)))
          io.imsave(QC_model_path+'/'+QC_model_name+'/Quality Control/RSE_GTvsSource_'+i, img_as_uint(np.clip(img_RSE_GTvsSource_32bit, -1, 1)))


      # -------------------------------- Calculate the RSE metrics and save them --------------------------------

      # Normalised Root Mean Squared Error (here it's valid to take the mean of the image)
        NRMSE_GTvsPrediction = np.sqrt(np.mean(img_RSE_GTvsPrediction))
        NRMSE_GTvsSource = np.sqrt(np.mean(img_RSE_GTvsSource))
        
      # We can also measure the peak signal to noise ratio between the images
        PSNR_GTvsPrediction = psnr(test_GT_norm,test_prediction_norm,data_range=1.0)
        PSNR_GTvsSource = psnr(test_GT_norm,test_source_norm,data_range=1.0)

        writer.writerow([i,str(index_SSIM_GTvsPrediction),str(index_SSIM_GTvsSource),str(NRMSE_GTvsPrediction),str(NRMSE_GTvsSource),
                         str(PSNR_GTvsPrediction),str(PSNR_GTvsSource)])

# All data is now processed saved
Test_FileList = os.listdir(prediction_QC_folder) # this assumes, as it should, that both source and target are named the same


plt.figure(figsize=(15,15))
# Currently only displays the last computed set, from memory

df = pd.read_csv(QC_model_path+"/"+QC_model_name+"/Quality Control/QC_metrics_"+QC_model_name+".csv")

# ------------- For display ------------
print('------------------------------------------------------------------------------------------------------------------------')
print("Choose the image file (once selected, the new set of images will take a few seconds to appear):")
print(" ")
@interact
def show_QC_results(file = sorted(os.listdir(Source_QC_folder))):
  visualise_image_comparison_QC(image = file, dimension='2D', Source_folder=Source_QC_folder , Prediction_folder= prediction_QC_folder, Ground_truth_folder=Target_QC_folder, QC_folder=QC_model_path+"/"+QC_model_name+"/Quality Control", QC_scores= df )  

print('------------------------------------------------------------------------------------------------------------------------')

# **6. Using the trained model**

---

<font size = 4>In this section the unseen data is processed using the trained model (in section 4). First, your unseen images are uploaded and prepared for prediction. After that your trained model from section 4 is activated and finally saved into your Google Drive.

## **6.1. Generate prediction(s) from unseen dataset**
---
<font size = 4>The current trained model (from section 4.2) can now be used to process images. If you want to use an older model, untick the **Use_the_current_trained_model** box and enter the name and path of the model to use. Predicted output images are saved in your **Result_folder** folder as restored image stacks (ImageJ-compatible TIFF images).

<font size = 4>**`Data_folder`:** This folder should contain the images that you want to use your trained network on for processing.

<font size = 4>**`Result_folder`:** This folder will contain the predicted output images.

In [ ]:
#@markdown ### Provide the path to your dataset and to the folder where the predictions are saved, then play the cell to predict outputs from your unseen images.

Data_folder = "" #@param {type:"string"}
Result_folder = "" #@param {type:"string"}

#@markdown ###Does **Data_folder** contain *High resolution* images?
#@markdown ##### If your data only contains *High resolution* images, check the option below. The *Low resolution* images will be generated from the *High resolution* data given in the **Data_foder**.    

Are_high_resolution_images = False #@param {type:"boolean"}

Type_of_data = "Electron microscopy" #@param ["Electron microscopy", "Fluorescence"]

#@markdown ##### Aside from the predicted data, if it is desired, the generated *Low resolution* images can be saved checking the option below and providing the desired folder where the *Low resolution* data will be saved.

Save_generated_low_resolution_images = False #@param {type:"boolean"}
Low_resolution_folder = "" #@param{type:"string"}

# model name and path
#@markdown ###Do you want to use the current trained model?
Use_the_current_trained_model = True #@param {type:"boolean"}

#@markdown ###If not, provide the name of the model and path to model folder:
#@markdown #####During training, the model files are automatically saved inside a folder named after model_name in section 3. Provide the path to this folder below.
Prediction_model_folder = "" #@param {type:"string"}
Prediction_down_factor = 2 #@param {type:"number"}


if not os.path.exists(Result_folder):
  print("The "+Result_folder+ " path does not exist, it will be generated.")
  os.makedirs(Result_folder)

#Here we find the loaded model name and parent path
Prediction_model_name = os.path.basename(Prediction_model_folder)
Prediction_model_path = os.path.dirname(Prediction_model_folder)

if (Use_the_current_trained_model): 
  print("Using current trained network.")
  Prediction_model_name = Model_name
  Prediction_model_path = Model_path
else:
  Down_factor = Prediction_down_factor
  Initial_learning_rate = 0.001

full_Prediction_model_path = os.path.join(Prediction_model_path, Prediction_model_name)
if os.path.exists(full_Prediction_model_path):
  print("The "+Prediction_model_name+" network will be used.")
else:
  W  = '\033[0m'  # white (normal)
  R  = '\033[31m' # red
  print(R+'!! WARNING: The chosen model does not exist !!'+W)
  print('Please make sure you provide a valid model path and model name before proceeding further.')

# Read the list of file names
_, extension = os.path.splitext(os.listdir(Data_folder)[0])

filenames = [x for x in os.listdir( Data_folder ) if x.endswith(extension)]
filenames.sort()
print( 'Available images: ' + str( len(filenames)) )

images = [img_as_float32( io.imread( Data_folder + '/' + fil ) ) for fil in filenames]

if Are_high_resolution_images:
  if Type_of_data == "Electron microscopy":
    images = [em_crappify(x, Down_factor) for x in images]
  else:
    images = [fluo_crappify(x, Down_factor) for x in images]

  if Save_generated_low_resolution_images:
    if not os.path.exists(Low_resolution_folder):
      print("The "+Low_resolution_folder+ " path does not exist, it will be generated.")
      os.makedirs(Low_resolution_folder)
    for i, image  in enumerate(images):
      tf.keras.preprocessing.image.save_img(Low_resolution_folder+'/'+filenames[i], 
                                            np.expand_dims(image,axis=-1), 
                                            data_format=None, file_format=None)

images = np.expand_dims(images, axis=-1)

# Create new model using new input shape (rest of parameters are the same)

# Custom WDSR B model (0.62M parameters)
model = wdsr_b(scale=Down_factor, num_res_blocks=32)

optim = tf.keras.optimizers.Adam( learning_rate=Initial_learning_rate )
ç
loss_funct = 'mean_absolute_error' # [ms_ssim_loss(max_val=1)]
eval_metric = 'mean_squared_error'

# compile the model with RMSprop as optimizer, MSE as loss function and MAE as metric
model.compile(optimizer=optim, loss=loss_funct, metrics=[eval_metric])

# Load old weights
model.load_weights( os.path.join(full_Prediction_model_path, 'weights_best.h5') )

print( 'Predicting...' )

predictions = model.predict(images, batch_size=8)

for i, image  in enumerate(predictions):
  tf.keras.preprocessing.image.save_img(Result_folder+'/'+filenames[i], image, data_format=None, file_format=None)

print("Images saved into folder:", Result_folder)

## **6.2. Inspect the predicted output**
---



In [ ]:
# @markdown  ##Run this cell to see some inputs with their corresponding outputs
##Run this cell to display a randomly chosen input and its corresponding predicted output.

# ------------- For display ------------
print('---------------------------------------------------------------------------------------------------------------------------------------------------')
print("Choose the image file (Once selected, the new set of images will take a few seconds to appear):")
print(" ")
@interact
def show_prediction_results(file = sorted(os.listdir(Data_folder))):
  imageHR = io.imread(os.path.join(Data_folder, file))
  imageP = io.imread(os.path.join(Result_folder, file))

  plt.figure(figsize=(25,25))
  plt.subplot(3, 3, 1)
  plt.imshow( imageHR, 'gray' )
  if Are_high_resolution_images:
    plt.title( 'High resolution' )
  else:
    plt.title( 'Low resolution' )
  
  # ant its prediction
  plt.subplot(3, 3, 2)
  if Save_generated_low_resolution_images:
    imageLR = io.imread(os.path.join(Low_resolution_folder, file))
    plt.imshow( imageLR, 'gray' )
    plt.title( 'Low resolution' ) 
    plt.subplot(3, 3, 3)
    plt.imshow( imageP, 'gray' )
    plt.title( 'Prediction' ) 
  else:
    plt.imshow( imageP, 'gray' )
    plt.title( 'Prediction' ) 

print('---------------------------------------------------------------------------------------------------------------------------------------------------')


## **6.3. Download your predictions**
---

<font size = 4>**Store your data** and ALL its results elsewhere by downloading it from Google Drive and after that clean the original folder tree (datasets, results, trained model etc.) if you plan to train or use new networks. Please note that the notebook will otherwise **OVERWRITE** all files which have the same name.

# **7. Version log**
---




#**Thank you for using WGAN!**